## Assess the significance

In [22]:
import sys
import pandas as pd
import numpy as np
from scipy.stats import norm

In [3]:
def get_SNP_scores(filename, bind_scaler=0,dirsupt_scaler_pos=0,dirsupt_scaler_neg=0):
    bind_scores=list()
    disrupt_scores=list()
    svm_df=pd.read_csv(filename,sep="\t",header=0)
    bindref_column=svm_df['binding_ref'].tolist()
    bindalt_column=svm_df['binding_alt'].tolist()
    delta_column=svm_df['deltaSVM_score'].tolist()
    n=int(len(delta_column))
    
    for i in range(0,n):
        a=bindref_column[i]
        b=bindalt_column[i]
        bind_score = max(a,b)
        if bind_score>=0:
            bind_scores.append(bind_score)
            disrupt_score=delta_column[i]
            disrupt_scores.append(disrupt_score)
    if bind_scaler==0:
        bind_scaler=max(bind_scores)
        dirsupt_scaler_pos=max([i for i in disrupt_scores if i >= 0] or None)
        dirsupt_scaler_neg=min([i for i in disrupt_scores if i < 0] or None)
    scaled_bind_scores=list(np.divide(np.array(bind_scores),bind_scaler))
    scaled_disrupt_scores=list()
    for score in disrupt_scores:
        if score>=0:
            newscore=score/dirsupt_scaler_pos
        else:
            newscore=-score/dirsupt_scaler_neg
        scaled_disrupt_scores.append(newscore)
    scaled_mod_scores=list(np.multiply(scaled_bind_scores,scaled_disrupt_scores))
    return bind_scaler,dirsupt_scaler_pos,dirsupt_scaler_neg,scaled_bind_scores,scaled_disrupt_scores,scaled_mod_scores

In [4]:
target_filename="GATA1_t.scores"
background_filename="GATA1_bg.scores"

In [5]:
bind_scaler,dirsupt_scaler_pos,dirsupt_scaler_neg,scaled_bind_scores_bg,scaled_disrupt_scores_bg,scaled_mod_scores_bg \
    = get_SNP_scores(background_filename)

In [6]:
bind_scaler,dirsupt_scaler_pos,dirsupt_scaler_neg,scaled_bind_scores_target,scaled_disrupt_scores_target,scaled_mod_scores_target \
    = get_SNP_scores(target_filename,bind_scaler,dirsupt_scaler_pos,dirsupt_scaler_neg)

In [ ]:
target_data=scaled_mod_scores_target
bg_data=scaled_mod_scores_bg

pop_mean=np.mean(bg_data)
pop_var=np.var(bg_data)
test_mean=pop_mean

test_var=pop_var/(len(target_data))


t=(np.mean(target_data)-test_mean)/((test_var)**(0.5))
report_pvalue_left=norm.cdf(t)
report_pvalue_right=1-report_pvalue_left
#print(str(test_mean))
#print(str(test_var)) 
abs_target_data=np.abs(target_data)
abs_bg_data=np.abs(bg_data)
abs_pop_mean=np.mean(abs_bg_data)
abs_pop_var=np.var(abs_bg_data)
abs_test_mean=abs_pop_mean
abs_test_var=abs_pop_var/(len(abs_target_data))
if abs_test_var==0:
    abs_test_var=1E-20
abs_t=(np.mean(abs_target_data)-abs_test_mean)/((abs_test_var)**(0.5))
report_pvalue_abs=1-norm.cdf(abs_t)
print("Disruption p-value: "+str(report_pvalue_left))
print("Enhancement p-value: "+str(report_pvalue_right))
print("Dual p-value: "+str(report_pvalue_abs))